In [18]:
import pandas as pd
from Bio.Data.IUPACData import protein_letters_3to1 as AA_CONVERTER
from abnumber import Chain
from utils import read_pdb_line, get_sabdab_details, remove_redundant
import numpy as np
from scipy.spatial import distance_matrix
from multiprocessing import Pool


class Structure:
    def __init__(self, idcode, df_pdb):
        self.idcode = idcode
        self.pdb_path = f"{idcode}.pdb"

        self.chains = []
        for i, values in df_pdb.iterrows():
            chains = list(values[["Hchain", "Lchain"]].dropna().unique())
            if len(chains) == 2:
                self.chains += chains

        self.antigen_chains = df_pdb.antigen_chain.values[0].replace(" | ", "")

        self.get_sequence()
        self.ab_chains = {}

    def get_sequence(self):
        self.sequence = {chain: "" for chain in self.chains}
        self.seqnumbs = {chain: [] for chain in self.chains}
        last = None
        with open(self.pdb_path) as f:
            for line in f:
                if line.startswith("ATOM"):
                    chain, res, *_ = read_pdb_line(line)
                    (resname, resnumb) = res
                    new = resname, chain, resnumb
                    if chain in self.chains and last != new:
                        res = AA_CONVERTER[resname.capitalize()]
                        self.sequence[chain] += res
                        self.seqnumbs[chain].append(resnumb)
                    last = new[:]

    def get_cdrs(self, scheme):
        cdrs = {}
        for ichain in range(0, len(self.chains), 2):
            hchain, lchain = self.chains[ichain : ichain + 2]
            chains = {}
            for chainid in (hchain, lchain):
                seq = self.sequence[chainid]
                print(chainid, seq, scheme)
                try:
                    chain = Chain(seq, scheme=scheme)
                    chains[chainid] = chain
                except:
                    continue

            if len(chains) != 2:
                continue

            for chainid in (hchain, lchain):
                seq = self.sequence[chainid]
                chain = chains[chainid]
                ab_chain = AbChain(seq, chainid, chain.chain_type)
                self.ab_chains[chainid] = ab_chain

                cdrs[chainid] = {
                    1: chain.cdr1_seq,
                    2: chain.cdr2_seq,
                    3: chain.cdr3_seq,
                }
                for i, cdr in cdrs[chainid].items():
                    b = seq.index(cdr)
                    e = b + len(cdr) - 1

                    b_resnumb = self.seqnumbs[chainid][b]
                    e_resnumb = self.seqnumbs[chainid][e]

                    ab_chain.add_cdr(i, cdr, b_resnumb, e_resnumb)

    def get_cdr_epitope(self, cutoff):
        if not self.ab_chains:
            return

        cur_cdr = None
        cdr_i = None
        prev_resnumb = None
        # Assign atoms to CDRs
        with open(self.pdb_path) as f:
            for line in f:
                if line.startswith("ATOM"):
                    chain, res, atom, coords = read_pdb_line(line)
                    if chain in self.ab_chains:
                        (resname, resnumb) = res

                        abchain = self.ab_chains[chain]

                        if not cur_cdr:
                            for i, cdr in abchain.cdrs.items():
                                if resnumb >= cdr.i_begin and resnumb <= cdr.i_end:
                                    if resnumb == cdr.i_begin:
                                        cur_cdr = i
                                        cdr_i = -1
                                        break

                        if cur_cdr:
                            cdr = abchain.cdrs[cur_cdr]

                            if prev_resnumb != resnumb:
                                cdr_i += 1

                            if resnumb > cdr.i_end or cdr_i >= len(cdr.seq):
                                # print("Reset")
                                prev_resnumb = None
                                cur_cdr = None
                            else:
                                # print(
                                #     resnumb,
                                #     cdr.i_begin,
                                #     cdr.i_end,
                                #     resname,
                                #     AA_CONVERTER[resname.capitalize()],
                                #     cdr.seq[cdr_i],
                                #     cdr_i,
                                # )
                                assert (
                                    AA_CONVERTER[resname.capitalize()] == cdr.seq[cdr_i]
                                ), f"CDR from sequence does not match the structure\n{AA_CONVERTER[resname.capitalize()]}\n{cdr.seq[cdr_i]}"
                                cdr.add_atom(atom, res, coords)

                                prev_resnumb = resnumb

        candidates_details = []
        candidates_coords = []
        with open(self.pdb_path) as f:
            for line in f:
                if line.startswith("ATOM"):
                    chain, res, atom, coords = read_pdb_line(line)
                    if chain in self.antigen_chains:
                        (resname, resnumb) = res

                        candidates_details.append((chain, res, atom))
                        candidates_coords.append(coords)

        if len(candidates_coords) == 0:
            return

        residues = []
        for chainid, abchain in self.ab_chains.items():
            for cdr in abchain.cdrs.values():
                cdr_atoms = [atom.coords for atom in cdr.atoms.values()]

                to_print = {}
                for chain, res, atom in candidates_details:
                    if chain not in to_print:
                        to_print[chain] = []
                    to_print[chain].append(atom[1])

                if len(cdr_atoms):
                    dm = distance_matrix(candidates_coords, cdr_atoms)
                else:
                    continue
                cutoff_dm = dm < cutoff
                for i in range(dm.shape[0]):
                    if cutoff_dm[i].sum() > 0:
                        anumb = candidates_details[i][2][1]
                        resnumb = candidates_details[i][1][1]
                        cdr.save_epitope(anumb, resnumb)

                # cdr.print_pymol_selection()
                # exit()

    def print_cdrs(self):
        for chain in self.ab_chains.values():
            for i, cdr in chain.iter_cdrs():
                print(
                    chain.chain_id, chain.chain_type, i, cdr.seq, cdr.i_begin, cdr.i_end
                )

    def save_cdr_epitopes(self):
        cdrs = []
        for chain in self.ab_chains.values():
            for i, cdr in chain.iter_cdrs():
                cdrs.append(
                    [
                        self.idcode,
                        chain.chain_id,
                        chain.chain_type,
                        i,
                        cdr.seq,
                        cdr.i_begin,
                        cdr.i_end,
                        [atom.anumb for atom in cdr.atoms.values()],
                        cdr.epitope,
                        cdr.epitope_res,
                    ]
                )

        return cdrs


class AbChain:
    def __init__(self, seq, chain_id, chain_type):
        self.seq = seq
        self.chain_id = chain_id
        self.chain_type = chain_type

        self.cdrs = {}

    def add_cdr(self, cdr_type, seq, begin, end):
        cdr = CDR(cdr_type, seq, begin, end)
        self.cdrs[cdr_type] = cdr

    def iter_cdrs(self):
        for i, cdr in self.cdrs.items():
            yield i, cdr


class CDR:
    def __init__(self, cdr_type, seq, b, e):
        self.numb = cdr_type
        self.seq = seq
        self.i_begin = b
        self.i_end = e
        self.size = len(seq)
        self.atoms = {}
        self.epitope = []
        self.epitope_res = []

    def add_atom(self, atom, res, coords):
        new_atom = Atom(atom, res, coords)
        self.atoms[atom[1]] = new_atom

    def save_epitope(self, anumb, resnumb):
        self.epitope.append(anumb)
        self.epitope_res.append(resnumb)

    def print_pymol_selection(self):
        sel_cdr = "+".join([str(i) for i in self.atoms.keys()])
        sel_epitope = "+".join([str(i) for i in self.epitope])
        print("CDR:", len(self.atoms), sel_cdr)
        print("Epitope:", len(self.epitope), sel_epitope)


class Atom:
    def __init__(self, atom, res, coords):
        self.resname, self.resnumb = res
        self.aname, self.anumb = atom
        self.coords = np.array(coords)


def run(x):
    i, pdb = x
    print(i, len(unique_pdbs), pdb)
    df_pdb = df_details.query(f'pdb == "{pdb}"')[["Hchain", "Lchain", "antigen_chain"]]

    try:
        struct = Structure(pdb, df_pdb)
        struct.get_cdrs(scheme)
        struct.print_cdrs()
        struct.get_cdr_epitope(cutoff)
        cdrs = struct.save_cdr_epitopes()

    except:
        cdrs = None
        print("failed ->", pdb)

    if cdrs:
        df = pd.DataFrame(
            cdrs,
            columns=(
                "idcode",
                "chainID",
                "chain_type",
                "cdr",
                "cdr_seq",
                "cdr_begin",
                "cdr_end",
                "cdr_atoms",
                "epitope_atoms",
                "epitope_residues",
            ),
        )
        df, removed = remove_redundant(df)
        return df, removed
    else:
        print("skipped")


if __name__ == "__main__":
    df_details = get_sabdab_details()
    scheme = "chothia"
    cutoff = 5

    df = pd.DataFrame(
        columns=(
            "idcode",
            "chainID",
            "chain_type",
            "cdr",
            "cdr_seq",
            "cdr_begin",
            "cdr_end",
            "cdr_atoms",
            "epitope_atoms",
            "epitope_residues",
        ),
    )
    df_removed = df.copy()
    unique_pdbs = df_details.pdb.unique()

    df_r = run((0, "1a14"))
    # print(
    #     df_r[0][
    #         [
    #             "idcode",
    #             "chainID",
    #             "chain_type",
    #             "cdr",
    #             "cdr_atoms",
    #             "epitope_residues",
    #         ]
    #     ]
    # )
    # exit()

#     for i, pdb in enumerate(unique_pdbs[:10]):
#         print(f"### {i} ###")
#         df_r = run((i, pdb))
#        # print(df_r[["idcode", "chainID", "chain_type", "cdr", "epitope_residues"]])
#         exit()

    #with Pool(1) as p:
        #rs = p.map(run, [(i, pdb) for i, pdb in enumerate(unique_pdbs[:10])])
#     rs = list(map(run, [(i, pdb) for i, pdb in enumerate(unique_pdbs[1950:1960])]))
#     for r in rs:
#         if r is not None:
#             if r[0] is not None:
#                 df = df.append(r[0], ignore_index=True)
#             if r is not None or r[1] is not None:
#                 df_removed = df_removed.append(r[1], ignore_index=True)

#     print(f"PDBs: {len(df.idcode.unique())}\tCDRs: {len(df)}\tAbAg: {len(df)/6}")
#     df.to_pickle("../data/cdr_epitope.pickle")

#     print(
#         f"PDBs: {len(df_removed.idcode.unique())}\tCDRs: {len(df_removed)}\tAbAg: {len(df_removed)/6}"
#     )
   # df_removed.to_pickle("../data/cdr_epitope_redundant.pickle")

SabDab
AbAg Complexes: 6252
PDB Files: 3123

0 3123 1a14
H QVQLQQSGAELVKPGASVRMSCKASGYTFTNYNMYWVKQSPGQGLEWIGIFYPGNGDTSYNQKFKDKATLTADKSSNTAYMQLSSLTSEDSAVYYCARSGGSYRYDGGFDYWGQGTTVTV chothia
L DIELTQTTSSLSASLGDRVTISCRASQDISNYLNWYQQNPDGTVKLLIYYTSNLHSEVPSRFSGSGSGTDYSLTISNLEQEDIATYFCQQDFTLPFTFGGGTAA chothia
H H 1 GYTFTNY   26    32 
H H 2 YPGNGD   52    56 
H H 3 SGGSYRYDGGFDY   95   102 
L K 1 RASQDISNYLN   24    34 
L K 2 YTSNLHS   50    56 
L K 3 QQDFTLPFT   89    97 


In [ ]:
https://github.com/jamie-alnasir/artemis/blob/master/Artemis.py#L67

In [2]:
for i,s in enumerate(unique_pdbs):
    #print(i,s)
    if str(s) =="1a14":
        print(i)

1952


In [20]:
df_r[0]

,idcode,chainID,chain_type,cdr,cdr_seq,cdr_begin,cdr_end,cdr_atoms,epitope_atoms,epitope_residues
0,1a14,H,H,1,GYTFTNY,26,32,"[3297, 3298, 3299, 3300, 3301, 3302, 3303, 330...",[],[]
1,1a14,H,H,2,YPGNGD,52,56,"[3560, 3561, 3562, 3563, 3564, 3565, 3566, 356...","[2240, 2253, 2254, 2255, 2258, 2259, 2275, 227...","[ 367 , 369 , 370 , 370 , 370 , 370 , 37..."
2,1a14,H,H,3,SGGSYRYDGGFDY,95,102,"[4008, 4009, 4010, 4011, 4012, 4013, 4014, 401...","[1959, 1960, 1961, 2231, 2233, 2234, 2239, 224...","[ 329 , 329 , 329 , 366 , 366 , 366 , 36..."
3,1a14,L,K,1,RASQDISNYLN,24,34,"[4378, 4379, 4380, 4381, 4382, 4383, 4384, 438...","[1955, 1956, 1957, 1958, 1959, 1960, 1961, 196...","[ 329 , 329 , 329 , 329 , 329 , 329 , 32..."
4,1a14,L,K,2,YTSNLHS,50,56,"[4597, 4598, 4599, 4600, 4601, 4602, 4603, 460...",[1983],[ 332 ]
5,1a14,L,K,3,QQDFTLPFT,89,97,"[4897, 4898, 4899, 4900, 4901, 4902, 4903, 490...","[1938, 1939, 1947, 1948, 1949, 1950, 1951, 195...","[ 327 , 327 , 328 , 328 , 328 , 328 , 32..."


In [3]:
df_details = get_sabdab_details()
unique_pdbs = df_details.pdb.unique()

SabDab
AbAg Complexes: 6252
PDB Files: 3123



In [4]:
df_r[1]

,idcode,chainID,chain_type,cdr,cdr_seq,cdr_begin,cdr_end,cdr_atoms,epitope_atoms,epitope_residues


In [5]:
class Structure:
    def __init__(self, idcode, df_pdb):
        self.idcode = idcode
        self.pdb_path = f"{idcode}.pdb"

        self.chains = []
        for i, values in df_pdb.iterrows():
            chains = list(values[["Hchain", "Lchain"]].dropna().unique())
            if len(chains) == 2:
                self.chains += chains

        self.antigen_chains = df_pdb.antigen_chain.values[0].replace(" | ", "")

        self.get_sequence()
        self.ab_chains = {}

In [6]:
df_details = get_sabdab_details()

SabDab
AbAg Complexes: 6252
PDB Files: 3123



In [7]:
import pandas as pd
import os

#curdir = os.path.dirname(__file__)


def get_sabdab_details():
    f = "20230207_0460443_summary.tsv"
    df = pd.read_csv(f, sep="\t").drop_duplicates()
    #df = df.query(
    #    "antigen_type == antigen_type and antigen_type.str.contains('protein')"
    #)
    df = df.query("Hchain != Lchain")
    df = df.query("Hchain == Hchain and Lchain == Lchain")
    print(f"SabDab\nAbAg Complexes: {len(df)}\nPDB Files: {len(set(df.pdb.values))}\n")
    return df

In [ ]:
get_sabdab_details()

In [8]:
 f = "20230207_0460443_summary.tsv"
df = pd.read_csv(f, sep="\t").drop_duplicates()

In [ ]:
df.head()

In [9]:
df[df["pdb"] == "1a14"]

,pdb,Hchain,Lchain,model,antigen_chain,antigen_type,antigen_het_name,antigen_name,short_header,date,...,scfv,engineered,heavy_subclass,light_subclass,light_ctype,affinity,delta_g,affinity_method,temperature,pmid
3935,1a14,H,L,0,N,protein,NaN,neuraminidase,COMPLEX (ANTIBODY/ANTIGEN),12/21/97,...,False,True,IGHV1,IGKV10,Kappa,None,None,None,None,None


In [10]:
df = df.query("Hchain != Lchain")
df = df.query("Hchain == Hchain and Lchain == Lchain")
print(f"SabDab\nAbAg Complexes: {len(df)}\nPDB Files: {len(set(df.pdb.values))}\n")

SabDab
AbAg Complexes: 6252
PDB Files: 3123



In [11]:
s = Structure("1a14",)

TypeError: Structure.__init__() missing 1 required positional argument: 'df_pdb'

In [12]:
if __name__ == "__main__":
    df_details = get_sabdab_details()
    scheme = "chothia"
    cutoff = 5

    df = pd.DataFrame(
        columns=(
            "idcode",
            "chainID",
            "chain_type",
            "cdr",
            "cdr_seq",
            "cdr_begin",
            "cdr_end",
            "cdr_atoms",
            "epitope_atoms",
            "epitope_residues",
        ),
    )
    df_removed = df.copy()
    unique_pdbs = df_details.pdb.unique()

    df_r = run((0, "1a14"))

SabDab
AbAg Complexes: 6252
PDB Files: 3123

0 3123 1a14
failed -> 1a14
skipped


In [17]:
df_r = run((0, "1a14"))

0 3123 1a14
failed -> 1a14
skipped


In [13]:
df_r[0].loc[1,"epitope_residues"]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
df_r[0].loc[1,"epitope_atoms"]

In [15]:
df_r

In [1]:
df_removed

NameError: name 'df_removed' is not defined

In [2]:
df

NameError: name 'df' is not defined

In [4]:
unique_pdbs

NameError: name 'unique_pdbs' is not defined